[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

In [ ]:
!pip install geopandas osmnx rasterio
# IMPORTANT: Restart runtime after installation!!!

In [ ]:
import math
import urllib.request
import os
import glob
import subprocess
import shutil
import random
import matplotlib.pyplot as plt

import pandas as pd
import geopandas as gpd
import osmnx as ox
import rasterio 
from rasterio import plot
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.merge import merge
from rasterio.windows import Window
from osgeo import gdal 
import ee

## Trigger the authentication flow. You only need to do this once
## To be able to access google earth engine you have to register first
ee.Authenticate()
# Initialize the library.
ee.Initialize()

In [ ]:
############################################################################################################################
# Really smart code to get merged geotiff tiles from gps coordinates                                                       #
# got from here: https://dev.to/jimutt/generate-merged-geotiff-imagery-from-web-maps-xyz-tile-servers-with-python-4d13     #
############################################################################################################################

from math import log, tan, radians, cos, pi, floor, degrees, atan, sinh
from rasterio.transform import from_origin

def sec(x):
    return(1/cos(x))

def latlon_to_xyz(lat, lon, z):
    tile_count = pow(2, z)
    x = (lon + 180) / 360
    y = (1 - log(tan(radians(lat)) + sec(radians(lat))) / pi) / 2
    return(tile_count*x, tile_count*y)

def bbox_to_xyz(lon_min, lon_max, lat_min, lat_max, z):
    x_min, y_max = latlon_to_xyz(lat_min, lon_min, z)
    x_max, y_min = latlon_to_xyz(lat_max, lon_max, z)
    return(floor(x_min), floor(x_max),
           floor(y_min), floor(y_max))

def mercatorToLat(mercatorY):
    return(degrees(atan(sinh(mercatorY))))

def y_to_lat_edges(y, z):
    tile_count = pow(2, z)
    unit = 1 / tile_count
    relative_y1 = y * unit
    relative_y2 = relative_y1 + unit
    lat1 = mercatorToLat(pi * (1 - 2 * relative_y1))
    lat2 = mercatorToLat(pi * (1 - 2 * relative_y2))
    return(lat1, lat2)

def x_to_lon_edges(x, z):
    tile_count = pow(2, z)
    unit = 360 / tile_count
    lon1 = -180 + x * unit
    lon2 = lon1 + unit
    return(lon1, lon2)

def tile_edges(x, y, z):
    lat1, lat2 = y_to_lat_edges(y, z)
    lon1, lon2 = x_to_lon_edges(x, z)
    return[lon1, lat1, lon2, lat2]

def download_tile(x, y, z, tile_server):
    url = tile_server.replace(
        "{x}", str(x)).replace(
        "{y}", str(y)).replace(
        "{z}", str(z))
    path = f'{temp_dir}/{x}_{y}_{z}.png'
    urllib.request.urlretrieve(url, path)
    return(path)

def merge_tiles(input_pattern, output_path):
    merge_command = ['gdal_merge.py', '-o', output_path]
    for name in glob.glob(input_pattern):
        merge_command.append(name)
    subprocess.call(merge_command)

def georeference_raster_tile(x, y, z, path):
    bounds = tile_edges(x, y, z)
    filename, extension = os.path.splitext(path)
    gdal.Translate(filename + '.tif',
                   path,
                   outputSRS='EPSG:4326',
                   outputBounds=bounds)


###################################################################################################################################
#  code to get geotiff urls from any google earth engine dataset, based on "Browse Earth Engine Datasets with folium" experiment  #
###################################################################################################################################

# get url template based on required dataset name 
def get_tiles_url(dataset_name, start_date = '2016-01-01', end_date = '2016-12-31'):
    url_template = ''
    if dataset_name == 'ee_fnf':
      url_template = get_ee_tiles_url(FNF, start_date, end_date)
    elif dataset_name == 'ee_landsat8':
      url_template = get_ee_tiles_url(Landsat8, start_date, end_date)
    elif dataset_name == 'ee_sentinel2':
      url_template = get_ee_tiles_url(Sentinel2, start_date, end_date)
    elif dataset_name == 'google':
      url_template = "https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}"
    elif dataset_name == 'mapbox':  
      url_template = "https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}.png?access_token=pk.eyJ1IjoiaGJobWxhaSIsImEiOiJja2x4MHcxb3gybzZ6Mm5ud2pyNDgxNWJ1In0.2LGg2fkl5yUTv2mUpuQCug"
    else:
      url_template = "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
    return url_template
  
# get earth engine url template based on required dataset class 
def get_ee_tiles_url(ee_class, start_date, end_date):
    ee_class.start_date = start_date
    ee_class.end_date = end_date
    return ee.Image(ee_class.ee_object.mosaic()).getMapId(ee_class.vis_params)['tile_fetcher'].url_format

class FNF:
#https://developers.google.com/earth-engine/datasets/catalog/JAXA_ALOS_PALSAR_YEARLY_FNF
# data from 2007 to 2017:
  name = 'Global PALSAR-2/PALSAR Forest/Non-Forest Map -2017'
  start_date = '2016-01-01'
  end_date = '2016-12-31'
  ee_object = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/FNF').filterDate(start_date, end_date).select('fnf')
  vis_params = {
    'min': 1.0,
    'max': 3.0,
    'palette': ['006400', 'FEFF99', '0000FF'],
  }

class Landsat8:
#https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR#description
  name = "USGS Landsat 8 Surface Reflectance Tier 1"
  start_date = '2016-01-01'
  end_date = '2016-12-31'
  def maskL8sr(image):
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)
  ee_object = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate(start_date, end_date).filter(ee.Filter.lt('CLOUD_COVER',2)).map(maskL8sr)
  vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
  }

class Sentinel2:
  #https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR?hl=en
  name = 'Sentinel-2 MSI: MultiSpectral Instrument, Level-2A'
  start_date = '2019-01-01'
  end_date = '2019-12-31'
  def maskS2clouds(image):
    qa = image.select('QA60')
    #Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)
  ee_object = ee.ImageCollection('COPERNICUS/S2_SR').filterDate(start_date, end_date).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',2)).map(maskS2clouds)
  vis_params = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
  }

# main function to download images
def download_image(place_name=False, gps_box=False, tile_server='ee_sentinel2', zoom=12, start_date = '2019-01-01',  end_date = '2019-12-31', tile_limit = 1000, temp_dir = 'temp', output_dir = ''):
  if tile_server[:3] == 'ee_':
    output_file = os.path.join(output_dir, f'{place_name if place_name else str(gps_box)}, {tile_server}, zoom:{str(zoom)}, from:{start_date} to:{end_date}.tif')
  else:
    output_file = os.path.join(output_dir, f'{place_name if place_name else str(gps_box)}, {tile_server}, zoom:{str(zoom)}.tif')

  if place_name:
    # search the place area, based on the given name  
    place_area = ox.geocoder.geocode_to_gdf(place_name)
    print('Got the following place: ' + place_area.display_name[0])
    bounds = place_area.bounds
    lon_min, lon_max, lat_min, lat_max = bounds.minx.values[0], bounds.maxx.values[0], bounds.miny.values[0], bounds.maxy.values[0]
    print('Downloading from the following GPS coordinates')
    print(bounds)
  elif gps_box:
    lon_min, lon_max, lat_min, lat_max = gps_box
    print('Downloading from the following GPS coordinates')
    print(gps_box)
  else:
    print('please define place_name or gps_box')
    return

  if not os.path.exists(temp_dir):
      os.makedirs(temp_dir)

  x_min, x_max, y_min, y_max = bbox_to_xyz(
      lon_min, lon_max, lat_min, lat_max, zoom)
  tile_number = (x_max - x_min + 1) * (y_max - y_min + 1)

  if tile_number > tile_limit:
    print('Nº of tiles above 1000, please select a smaller area or zoom')
    return
  else:
    print(f"Downloading {tile_number} tiles from {tile_server}, with a zoom of {zoom}")
    for x in range(x_min, x_max + 1):
        for y in range(y_min, y_max + 1):
            png_path = download_tile(x, y, zoom, get_tiles_url(tile_server, start_date, end_date))
            print(f"got tile nº {x}x{y}")
            georeference_raster_tile(x, y, zoom, png_path)
    print("Download complete")
    print("Merging tiles")
    merge_tiles(temp_dir + '/*.tif', output_file)
    print(f"Merged image saved in {output_file}")
    shutil.rmtree(temp_dir)
    return output_file

def view_image(image_path):
  print('9 random images at maximum resolution')
  dataset = rasterio.open(image_path)
  fig, plot_list = plt.subplots(3,3, figsize=(30,30))
  for plt1 in plot_list:
    for plt2 in plt1:
      random_plot(dataset, plt2)
  plt.show()
  print('Full image:')
  show(dataset)

def random_plot(src, my_plt, xsize= 256, ysize = 256):
    # xsize, ysize: The size in pixels of your desired window
    # Generate a random window location that doesn't go outside the image
    xmin, xmax = 0, src.width - xsize
    ymin, ymax = 0, src.height - ysize
    xoff, yoff = random.randint(xmin, xmax), random.randint(ymin, ymax)

    # Create a Window and calculate the transform from the source dataset    
    cropped = src.read(window=Window(xoff, yoff, xsize, ysize))
    show(cropped, ax=my_plt)

In [ ]:
#---------- CONFIGURATION -----------#
# Choose any place worlwide you want (like city, country)
place_name = "Ericeira, Portugal"

# OR define GPS coordinates to retrive image  as [lon_min, lon_max, lat_min, lat_max] to get small areas
#gps_box = [-9.42, -9.41, 38.95,38.96]
#place_name = False

# Choose the tile server: 
# 'ersi': High resolution satellite images from ERSI, slower but higher download limits
# 'mapbox': High resolution satellite images from Mapbox, slower but higher download limits, needs registration (using my ID here)
# 'google' : Fastest and high resolution satellite images from google maps, but with limits to nº of downloads. 
#            If you reach the limit, factory reset the colab to be able to download again from google
# 'ee_sentinel2' : Sentinel-2 satellite dataset, Level-2A, from earth engine, already filtered with cloud < 2% and by date.
# 'ee_landsat8' : Landsat-8 satellite dataset from earth engine, already filtered with cloud < 2% and by date
# 'ee_fnf' : Florest/NonFlorest dataset from earth engine. I belive it can be usefull to label our images to train the model
tile_server = 'ersi' 

# Choose zoom:
#  for high resolution images of cities (ersi, mapbox or google server) use 12-18, depending on  the time you have to wait and choosed area
#  for earth engine datasets (landsat8, sentinel2 or other) use up to 14, no further resolution available
zoom = 16  

# choose image dates,  only works on ee datasets and in some dates, check documentation
start_date = '2019-01-01'
end_date = '2019-12-31'

tile_limit = 1000 #limit to avoid acidental time consuming downloads, increase if you need but colab may reset 

# Image file is saved on the default folder, change to your gdrive folder to save permanently the file
output_dir = ''

#----------END OF CONFIGURATION -----------#

In [ ]:
# Now, runnig all the code...
image_path = download_image(
    place_name=place_name, 
    gps_box=gps_box, 
    tile_server=tile_server, 
    zoom=zoom, 
    start_date=start_date, 
    end_date=end_date, 
    tile_limit=tile_limit, 
    output_dir=output_dir)

if image_path:
  view_image(image_path)

In [ ]:
# view new set of small images
view_image(image_path)